## Questão A — Definição de *feature* e necessidade de tratamento

**Feature** é um atributo coletado, uma variável preditora que representa um aspecto observado de um fenômeno-alvo, o alvo por outro lado é chamado "target". Em termos formais, cada feature é um componente de um vetor que alimenta o algoritmo de Machine Learning.

### Por que tratar features antes da modelagem?

1. **Escalas incompatíveis** – Distâncias e gradientes ficam distorcidos; resolve-se com normalização ou padronização.  
2. **Tipos incompatíveis** – Algoritmos numéricos não processam categorias brutas; emprega‐se *encoding* para dados qualitativos.  
3. **Dados ausentes e ruído** – Valores nulos (*NaN*) e outliers prejudicam a convergência; usa-se imputação e *robust scaling*.  
4. **Redundância e vazamento** – Variáveis colineares ou com informação futura causam *overfitting*; eliminam-se via análise de correlação ou pipelines bem definidos.

---

## Questão B — Seleção de features úteis

| Método      | Mecanismo                                                         | Quando usar                                  | Limitações                                                         |
|-------------|-------------------------------------------------------------------|----------------------------------------------|--------------------------------------------------------------------|
| **Filtragem** | Avalia cada feature isoladamente por estatísticas univariadas (teste qui-quadrado, ANOVA-F, informação mútua). | Datasets muito grandes; pré-poda rápida.      | Ignora interações e pode manter redundância.                       |
| **Wrapper**   | Testa subconjuntos treinando um modelo (RFE, *forward/backward selection*). | Conjuntos médios; busca desempenho máximo para um algoritmo específico. | Alto custo computacional; risco de *overfitting* se validação fraca. |
| **Embedding** | Seleção ocorre dentro do treinamento (Lasso, Árvores, mecanismos de atenção). | Dados de alta dimensão; modelos que já calculam importâncias. | Transparência reduzida; dependente do modelo escolhido.            |

**Estratégia recomendada**  
1. Aplicar filtragem para descartar irrelevâncias óbvias.  
2. Utilizar wrapper ou embedding para refinar de acordo com o algoritmo-alvo.  
3. Empregar validação cruzada para confirmar ganho de generalização.

---

## Questão C — Escalares, vetores e espaços vetoriais em ML

| Conceito            | Definição resumida                                                                                   | Exemplo em Machine Learning                                                    |
|---------------------|------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------|
| **Escalar**         | Número real utilizado para ponderar vetores.                                                         | Taxa de aprendizagem (eta = 0,01) em *gradient descent*.                       |
| **Vetor**           | Sequência ordenada de escalares, representada como x = (x₁, …, xₙ).                                   | Vetor de 784 pixels (28 × 28) que descreve uma imagem no conjunto MNIST.       |
| **Espaço vetorial** | Conjunto V de vetores fechado sob adição e multiplicação por escalares, obedecendo a axiomas lineares.| O espaço R³⁰⁰ que contém todos os *word embeddings* de 300 dimensões (Word2Vec).|

Esses conceitos sustentam operações essenciais em ML, como cálculo de distâncias (distância Euclidiana entre vetores), projeções e otimização em espaços de alta dimensão.


In [10]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import FunctionTransformer, PowerTransformer, MinMaxScaler, StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline

In [7]:
# ---------------------------------------------------------------
# Questão 2 — Breast Cancer Dataset
# ---------------------------------------------------------------
# Bibliotecas


# ---------------------------------------------------------------
# Questão 2(a) — Identificar as features contínuas
# ---------------------------------------------------------------
dados = load_breast_cancer()
df = pd.DataFrame(dados.data, columns=dados.feature_names)

print("[2(a)] Identificação das features contínuas\n")
print(f"Dataset contém {df.shape[0]} amostras e {df.shape[1]} features contínuas.\n")

# Todas as 30 colunas deste dataset já são numéricas contínuas.
features_continuas = df.columns.tolist()
print("\n[2(a)] Features contínuas no dataset:")
for feat in features_continuas:
    print(f" • {feat}")

# ---------------------------------------------------------------
# Questão 2(b) — Discretização por bins fixos
# ---------------------------------------------------------------
# Selecionar duas features contínuas para o exemplo
features_selecionadas = ["mean radius", "mean texture"]
print("\n[2(b)] Discretização por bins fixos (4 intervalos de mesma largura)\n")

for coluna in features_selecionadas:
    # 4 intervalos de igual largura
    bins_fixos = pd.cut(df[coluna], bins=4, include_lowest=True)
    contagem_fixos = bins_fixos.value_counts(sort=False)

    print(f"--- {coluna} ---")
    print(contagem_fixos, "\n")

# ---------------------------------------------------------------
# Questão 2(c) — Discretização por bins variáveis (quantis)
# ---------------------------------------------------------------
print("[2(c)] Discretização por bins variáveis (quartis)\n")

for coluna in features_selecionadas:
    # 4 intervalos gerados pelos quartis (mesmo nº aproximado de amostras)
    bins_variaveis = pd.qcut(df[coluna], q=4, duplicates="drop")
    contagem_variaveis = bins_variaveis.value_counts(sort=False)

    print(f"--- {coluna} ---")
    print(contagem_variaveis, "\n")


[2(a)] Identificação das features contínuas

Dataset contém 569 amostras e 30 features contínuas.


[2(a)] Features contínuas no dataset:
 • mean radius
 • mean texture
 • mean perimeter
 • mean area
 • mean smoothness
 • mean compactness
 • mean concavity
 • mean concave points
 • mean symmetry
 • mean fractal dimension
 • radius error
 • texture error
 • perimeter error
 • area error
 • smoothness error
 • compactness error
 • concavity error
 • concave points error
 • symmetry error
 • fractal dimension error
 • worst radius
 • worst texture
 • worst perimeter
 • worst area
 • worst smoothness
 • worst compactness
 • worst concavity
 • worst concave points
 • worst symmetry
 • worst fractal dimension

[2(b)] Discretização por bins fixos (4 intervalos de mesma largura)

--- mean radius ---
mean radius
(6.959, 12.263]     191
(12.263, 17.545]    276
(17.545, 22.828]     90
(22.828, 28.11]      12
Name: count, dtype: int64 

--- mean texture ---
mean texture
(9.679, 17.102]     185
(17

In [8]:
# ---------------------------------------------------------------
#Questão 3 Breast Cancer Dataset — Normalização
# ---------------------------------------------------------------


# ---------------------------------------------------------------
# 3.A) Normalização personalizada via FunctionTransformer
#    (Min-Max Scaling 0-1)
# ---------------------------------------------------------------
def min_max_scaler(arr):
    """Aplica min-max scaling coluna a coluna."""
    mins = np.min(arr, axis=0)
    maxs = np.max(arr, axis=0)
    return (arr - mins) / (maxs - mins)

min_max_transformer = FunctionTransformer(min_max_scaler, feature_names_out=None)
X_minmax = pd.DataFrame(
    min_max_transformer.fit_transform(X.values),
    columns=data.feature_names,
)

# Exibir estatísticas de verificação
print("\n[2.A] Estatísticas após Min-Max Scaling")
print(
    pd.DataFrame(
        {
            "min": X_minmax.min(),
            "max": X_minmax.max(),
            "mean": X_minmax.mean(),
            "std": X_minmax.std(ddof=0),
        }
    ).round(3)
)

# ---------------------------------------------------------------
# 3.B) Normalização via PowerTransform (Yeo-Johnson + padronização)
# ---------------------------------------------------------------
power = PowerTransformer(method="yeo-johnson", standardize=True)
X_power = pd.DataFrame(
    power.fit_transform(X.values),
    columns=data.feature_names,
)

# Exibir estatísticas de verificação
print("\n[3.B] Estatísticas após PowerTransform (Yeo-Johnson)")
print(
    pd.DataFrame(
        {
            "mean": X_power.mean(),
            "std": X_power.std(ddof=0),
            "skewness": X_power.skew(),
        }
    ).round(3)
)



[2.A] Estatísticas após Min-Max Scaling
                         min  max   mean    std
mean radius              0.0  1.0  0.338  0.167
mean texture             0.0  1.0  0.324  0.145
mean perimeter           0.0  1.0  0.333  0.168
mean area                0.0  1.0  0.217  0.149
mean smoothness          0.0  1.0  0.395  0.127
mean compactness         0.0  1.0  0.261  0.162
mean concavity           0.0  1.0  0.208  0.187
mean concave points      0.0  1.0  0.243  0.193
mean symmetry            0.0  1.0  0.380  0.138
mean fractal dimension   0.0  1.0  0.270  0.149
radius error             0.0  1.0  0.106  0.100
texture error            0.0  1.0  0.189  0.122
perimeter error          0.0  1.0  0.099  0.095
area error               0.0  1.0  0.063  0.085
smoothness error         0.0  1.0  0.181  0.102
compactness error        0.0  1.0  0.174  0.134
concavity error          0.0  1.0  0.081  0.076
concave points error     0.0  1.0  0.223  0.117
symmetry error           0.0  1.0  0.178  0.116

In [ ]:
# ---------------------------------------------------------------
# Questão 4 — Breast Cancer Dataset: Normalização, Escalonamento 
#             e Regularização
# ---------------------------------------------------------------

# ---------------------------------------------------------------
# 4(a) Min-Max Scaling (0 – 1)
# ---------------------------------------------------------------
minmax = MinMaxScaler()
X_minmax = pd.DataFrame(minmax.fit_transform(X), columns=data.feature_names)

print("\n[4(a)] Estatísticas após Min-Max Scaling (0–1):")
print(
    pd.DataFrame(
        {
            "min": X_minmax.min(),
            "max": X_minmax.max(),
            "mean": X_minmax.mean(),
            "std": X_minmax.std(ddof=0),
        }
    ).round(3)
)

# ---------------------------------------------------------------
# 4(b) Standard Scaling (média 0, desvio-padrão 1)
# ---------------------------------------------------------------
std_scaler = StandardScaler()
X_std = pd.DataFrame(std_scaler.fit_transform(X), columns=data.feature_names)

print("\n[4(b)] Estatísticas após Standard Scaling:")
print(
    pd.DataFrame(
        {
            "mean": X_std.mean(),
            "std": X_std.std(ddof=0),
            "min": X_std.min(),
            "max": X_std.max(),
        }
    ).round(3)
)

# ---------------------------------------------------------------
# 4(c) Regularização L2 (normalização por vetor linha)
# ---------------------------------------------------------------
# Normalizer aplica normalização L2 em cada amostra (linha), 
# resultando em vetores de norma 1.
X, y = load_breast_cancer(return_X_y=True)
l2_normalizer = Normalizer(norm="l2")
X_l2 = pd.DataFrame(l2_normalizer.fit_transform(X), columns=data.feature_names)

# Conferir se a norma de cada vetor linha é 1
normas = np.linalg.norm(X_l2.values, ord=2, axis=1)
print("\n[4(c)] Verificação da norma L2 de cada amostra (deve ser 1.0):")
print(pd.Series(normas).describe().round(3))

# ---------------------------------------------------------------
# 4(d) Aplicação simultânea de transformações
# ---------------------------------------------------------------
pipe = make_pipeline(
    StandardScaler(),      # Escalonamento padrão
    Normalizer(norm="l2")  # Regularização L2 por amostra
)

# Ajusta ao conjunto de dados e transforma
X_transf = pipe.fit_transform(X)

# ---------------------------------------------------------------
# Comprovações rápidas
# ---------------------------------------------------------------
# 1. Norma L2 de cada amostra deve ser 1
normas = np.linalg.norm(X_transf, ord=2, axis=1)
print("Norma L2 mínima, média, máxima:", normas.min(), normas.mean(), normas.max())

# 2. As features foram padronizadas antes da normalização
#    (as médias após o passo L2 tendem a zero, mas não exatamente,
#     pois a normalização altera as escalas relativas)
print("Média das primeiras 5 features após transformação:",
      np.mean(X_transf, axis=0)[:5])
#
#Sim. As transformações podem ser encadeadas em um Pipeline do scikit-learn:
#from sklearn.pipeline import make_pipeline
#pipe = make_pipeline(
#    StandardScaler(),   # passo 1
#    Normalizer(norm="l2")  # passo 2
#)
#X_transf = pipe.fit_transform(X)
#A ordem importa: normalmente usa-se um escalonamento (MinMax ou Standard)
#seguido pela normalização L2 se o modelo exigir vetores de unidade (ex.
#k-NN baseado em similaridade de cosseno). A aplicação simultânea deve
#ser justificada pelo algoritmo ou pelo pré-processamento desejado.
#



[4(a)] Estatísticas após Min-Max Scaling (0–1):
                         min  max   mean    std
mean radius              0.0  1.0  0.338  0.167
mean texture             0.0  1.0  0.324  0.145
mean perimeter           0.0  1.0  0.333  0.168
mean area                0.0  1.0  0.217  0.149
mean smoothness          0.0  1.0  0.395  0.127
mean compactness         0.0  1.0  0.261  0.162
mean concavity           0.0  1.0  0.208  0.187
mean concave points      0.0  1.0  0.243  0.193
mean symmetry            0.0  1.0  0.380  0.138
mean fractal dimension   0.0  1.0  0.270  0.149
radius error             0.0  1.0  0.106  0.100
texture error            0.0  1.0  0.189  0.122
perimeter error          0.0  1.0  0.099  0.095
area error               0.0  1.0  0.063  0.085
smoothness error         0.0  1.0  0.181  0.102
compactness error        0.0  1.0  0.174  0.134
concavity error          0.0  1.0  0.081  0.076
concave points error     0.0  1.0  0.223  0.117
symmetry error           0.0  1.0  0.17

'\nSim. As transformações podem ser encadeadas em um Pipeline do scikit-learn:\n\nfrom sklearn.pipeline import make_pipeline\npipe = make_pipeline(\n    StandardScaler(),   # passo 1\n    Normalizer(norm="l2")  # passo 2\n)\nX_transf = pipe.fit_transform(X)\n\nA ordem importa: normalmente usa-se um escalonamento (MinMax ou Standard)\nseguido pela normalização L2 se o modelo exigir vetores de unidade (ex.\nk-NN baseado em similaridade de cosseno). A aplicação simultânea deve\nser justificada pelo algoritmo ou pelo pré-processamento desejado.\n'